# Heat flux calculation for the thermal manikin with change material in thickness from 5mm to 7mm

In [1]:
from ngsolve import *
import matplotlib.pyplot as plt
import numpy as np
from netgen.read_gmsh import ReadGmsh
from calculations import *
from ngsolve.webgui import Draw


### Import and refine mesh

In [2]:
ngmesh = ReadGmsh("../Mesh/InfCyl(5mmTo7p5mm)")

mesh = Mesh(ngmesh)
mesh.GetBoundaries()
for i in range(1):
    mesh.Refine()

Draw(mesh)
print(mesh.GetBoundaries())

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

('bottom', 'baby', 'bag', 'top')


### Parameters and heat coefficient calculations

In [5]:
D = (67.5*2+2*7.5)*1e-3

T_ambient = 4.7
Ts = 15.5
T_baby = 34


k = 0.032
eps = 0.6


h_rads = get_h_rad(Ts,T_ambient, eps)
h_convs = get_h_conv(D, Ts, T_ambient)
h = h_rads + h_convs
print("h_conv =", h_convs)
print("h_rad =", h_rads)

h_conv = 3.6494084933804176
h_rad = 3.0935837154568486


### Finding the temperature using finite element

In [ ]:

#define finite element space and trial and test functions
fes = H1(mesh, order = 3, dirichlet = "baby")
u = fes.TrialFunction()
v = fes.TestFunction()

# robin bc introduced in the biliniar and linear term of the equation - > here baby bc is dirichlet and bag bc is robin 
a = BilinearForm(k*grad(u)*grad(v)*dx + h*u*v*ds("bag")).Assemble()
f = LinearForm(h*T_ambient*v*ds("bag")).Assemble()

#print(f.vec)

gfu = GridFunction(fes)
#apply dirichlet bnd_cond:
bc_vals = {"baby": T_baby, "bag": T_ambient, "top": None, "bottom":None}

boundaries = CoefficientFunction([bc_vals[mat] for mat in mesh.GetBoundaries()])
gfu.Set(boundaries,definedon=mesh.Boundaries("baby|bag"))


Draw(gfu)

#calculate
res = f.vec - a.mat * gfu.vec
gfu.vec.data += a.mat.Inverse(freedofs=fes.FreeDofs()) * res

Draw (gfu)


#analyze flux
W, Area_inner, W_per_m2 = get_heat_loss(mesh, gfu, h_rads,h_convs, T_ambient)
                
print(f"Total heat loss W = {W:.3f} W")
print(f"Inner surface area= {Area_inner:.3f} m^2")
print(f"Heat flux per unit area = {W_per_m2:.3f} W/m^2")


WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

### Save result to paraview

In [5]:
vtk = VTKOutput(ma=mesh, coefs=[gfu], names=["temperature"], filename="InfCyl(5mmTo7p5mm)", subdivision=3)
vtk.Do()

'InfCyl(5mmTo7p5mm)'